<a href="https://colab.research.google.com/github/flaviopradoaquino/TCC_MBA_USP_ESALQ/blob/main/TCC_USP_Teste_Normalidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalação e Importação das Bibliotecas**

In [ ]:
!pip install prophet
!pip install -q yfinance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import yfinance as yf
import datetime
import plotly.express as px
from prophet.plot import plot_plotly, plot_components_plotly
from datetime import date, timedelta
from prophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error

# **Construção do Dataset**

In [ ]:
dateparse = lambda dates: datetime.datetime.strptime(dates, '%Y-%m-%d')

In [ ]:
from datetime import datetime 
dt_atual = date.today()
td = timedelta(1095)
dt_final = datetime(2023, 3, 3)

In [ ]:
### Data de referência menos 1095 dias
df = yf.download("BOVA11.SA", start=(dt_final-td))['Close']

### Data atual menos 1095 dias
# df = yf.download("BOVA11.SA", start=(dt_atual-td))['Close']

### Intervalo de datas fixas
# df = yf.download("BOVA11.SA", start="2020-03-03", end="2023-03-03")['Close']

In [ ]:
df

In [ ]:
df = df.reset_index()
df

In [ ]:
df.info()

In [ ]:
df_bova_temp = df[['Date', 'Close']].rename(columns = {'Date': 'ds', 'Close': 'y'})
df_bova_temp

In [ ]:
### Alocando somente a data e excluindo uma possível hora inclusa na importação dos dados
df_bova = pd.DataFrame({
  "ds": df_bova_temp['ds'].dt.date,
  "y":df_bova_temp['y']
})

# **Excluindo Outliers**

In [ ]:
dados = df_bova['y']
dados.describe()

In [ ]:
boxplot_bova = px.box(dados, y="y")
boxplot_bova.show()

In [ ]:
q1 = np.quantile(dados, .25, method="midpoint")
q2 = np.quantile(dados, .5, method="midpoint")
q3 = np.quantile(dados, .75, method="midpoint")
IIQ = q3 - q1

In [ ]:
### Limite inferior do boxplot
limite_inferior = q1 - 1.5 * IIQ
limite_inferior

In [ ]:
### Limite superior do boxplot
limite_superior = q3 + 1.5 * IIQ
limite_superior

In [ ]:
df_bova_lim_inf = df_bova[df_bova['y']>limite_inferior]
df_bova_clean = df_bova_lim_inf[df_bova_lim_inf['y']<limite_superior]
df_bova = df_bova_clean
df_bova


## **Teste de Normalidade**

In [ ]:
import scipy.stats as stats

In [ ]:
stats.probplot(df_bova['y'], dist="norm", plot=plt)
plt.title("Normal QQ plot")
plt.show()

Teste Shapiro-Wilk

CRITÉRIOS:

NÍVEL DE SIGNIFICÂNCIA DE 0,05 ou 5% (MAIS UTILIZADO)

Ho = distribuição normal p > 0,05

Ha = distribuição não normal p <= 0,05

In [ ]:
e, p = stats.shapiro(df_bova['y'])
print('Estatística de teste: {}'.format(e))
print('p-valor: {}'.format(p))

In [ ]:
import seaborn as sns
sns.distplot(df_bova['y'])